In [3]:
#2021.06.21. MON
#Hankyeong

##CH05. 서포트 벡터 머신(Support Vector Machine)
#00. 패키지 호출
import numpy as np 
import pandas as pd 
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

#01. hyper parameter를 default로 설정해 머신러닝하기.  
#(1) 데이터셋 불러오기
cancer = load_breast_cancer()
df_cancer = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df_cancer['target'] = cancer.target
df_cancer

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [4]:
#(2) 데이터셋 탐색하기.
df_cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [8]:
#(3) Target 데이터 분포 확인하기. 
df_cancer['target'].value_counts()

1    357
0    212
Name: target, dtype: int64

In [10]:
#(4) Feature 데이터를 표준화 scaling하기
scaler = StandardScaler()
cancer_std = scaler.fit_transform(cancer.data)

In [11]:
df_cancer = pd.DataFrame(cancer_std, columns=cancer.feature_names)
df_cancer

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,...,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,...,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,...,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,...,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,...,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,2.110995,0.721473,2.060786,2.343856,1.041842,0.219060,1.947285,2.320965,-0.312589,-0.931027,...,1.901185,0.117700,1.752563,2.015301,0.378365,-0.273318,0.664512,1.629151,-1.360158,-0.709091
565,1.704854,2.085134,1.615931,1.723842,0.102458,-0.017833,0.693043,1.263669,-0.217664,-1.058611,...,1.536720,2.047399,1.421940,1.494959,-0.691230,-0.394820,0.236573,0.733827,-0.531855,-0.973978
566,0.702284,2.045574,0.672676,0.577953,-0.840484,-0.038680,0.046588,0.105777,-0.809117,-0.895587,...,0.561361,1.374854,0.579001,0.427906,-0.809587,0.350735,0.326767,0.414069,-1.104549,-0.318409
567,1.838341,2.336457,1.982524,1.735218,1.525767,3.272144,3.296944,2.658866,2.137194,1.043695,...,1.961239,2.237926,2.303601,1.653171,1.430427,3.904848,3.197605,2.289985,1.919083,2.219635


In [12]:
#(5) train, test 데이터셋으 분할하기. 
X_train,X_test,y_train,y_test = train_test_split(
    cancer_std, cancer.target, stratify=cancer.target, test_size=0.2, random_state=2021 
)

In [13]:
#(6) SVC 모델 생성하기. 
svc = SVC()

#(7) 모델 학습하기.
svc.fit(X_train,y_train)

SVC()

In [14]:
#(8) 모델 예측 하기
svc_pred = svc.predict(X_test)
accuracy_score(y_test, svc_pred)

0.9824561403508771

In [15]:
#02. 하이퍼 파라미터를 조정해 머신러닝하기. 
#(1) SVC 모델의 하이퍼 파라미터 확인하기. 
svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [25]:
#(2) GridSearch를 위한 하이퍼파라미터 값 지정하기. 
params = {'C' : [0.1,0.5,1,5,10,100]}

#(3)  GridsearchCV() 메서드를 이용해 훈련 모델 할당하기. 
svc = SVC(random_state=2021)
svc_gscv = GridSearchCV(svc, param_grid=params, cv=10, scoring='accuracy') 

#(4) 모델 학습하기. 
svc_gscv.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(random_state=2021),
             param_grid={'C': [0.1, 0.5, 1, 5, 10, 100]}, scoring='accuracy')

In [26]:
#(5) 최적 하이퍼 파라미터 확인하기. 
svc_gscv.best_params_

{'C': 5}

In [27]:
#(6) 최적 파라미터에 대한 평가 점수 확인하기. 
svc_gscv.best_score_

0.9801932367149758

In [28]:
#(7) test 데이터셋으로 모형 예측 및 평가하기. 
best_clf = svc_gscv.best_estimator_
pred = best_clf.predict(X_test)
accuracy_score(y_test, pred)

0.9824561403508771

In [29]:
#PLUS. 추가적인 하이퍼파라미터 튜닝하기. 
#(1) GridSearch를 위한 하이퍼파라미터 값 지정하기. 
params = {'C' : [2,3,4,5,6,7,8,9]}

#(2) GridsearchCV() 메서드를 이용해 훈련 모델 할당하기. 
svc = SVC(random_state=2021)
svc_gscv = GridSearchCV(svc, param_grid=params, cv=10, scoring='accuracy') 

#(3) 모델 학습하기. 
svc_gscv.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(random_state=2021),
             param_grid={'C': [2, 3, 4, 5, 6, 7, 8, 9]}, scoring='accuracy')

In [42]:
#(4) 최적 하이퍼 파라미터 확인하기. 
svc_gscv.best_score_

0.9824154589371981

In [30]:
#(5) 최적 파라미터에 대한 평가 점수 확인하기. 
svc_gscv.best_params_

{'C': 6}

In [31]:
#(5) test 데이터셋으로 모형 예측 및 평가 하기. 
best_clf = svc_gscv.best_estimator_
pred = best_clf.predict(X_test)
accuracy_score(y_test, pred)

0.9824561403508771